<a href="https://colab.research.google.com/github/kobrue02/evaluating-llm-generated-nlu-data/blob/main/bin/notebooks/train_eval_nlu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/kobrue02/evaluating-llm-generated-nlu-data/
%cd evaluating-llm-generated-nlu-data

Cloning into 'evaluating-llm-generated-nlu-data'...
remote: Enumerating objects: 823, done.
remote: Counting objects: 100% (232/232), done.
remote: Compressing objects: 100% (162/162), done.
remote: Total 823 (delta 158), reused 123 (delta 70), pack-reused 591 (from 1)
Receiving objects: 100% (823/823), 4.72 MiB | 7.55 MiB/s, done.
Resolving deltas: 100% (540/540), done.
/content/evaluating-llm-generated-nlu-data


In [2]:
from bin.framework.nlu_model import IntentClassifier
from bin.utils.methods import *

In [3]:
model = IntentClassifier()
datasets = [
    "zero_shot_simple_data",
    "one_shot_simple_data",
    "few_shot_simple_data",
    "chain_of_thought_simple_data",
    "persona_based_prompt_s1_data"
]

In [4]:
for fname in datasets:
    df = load_df(fname)
    train_df, test_df = model.split_dataset(df)
    model.fit(train_df)
    report = model.evaluate(test_df)
    print(fname)
    print(report)

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


zero_shot_simple_data
                                           precision    recall  f1-score   support

                    account_activate_item       0.23      0.60      0.33         5
       account_activate_item_not_possible       0.80      0.80      0.80         5
               account_ask_item_available       1.00      0.20      0.33         5
                      account_delete_item       0.23      1.00      0.37         5
                         account_get_item       1.00      0.60      0.75         5
                    account_login_problem       1.00      0.40      0.57         5
                            account_merge       1.00      1.00      1.00         5
account_setup_configure_item_not_possible       1.00      1.00      1.00         5
                        account_terminate       1.00      1.00      1.00         5
                           account_verify       0.75      0.60      0.67         5
                    account_verify_failed       0.60      0.60  

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


one_shot_simple_data
                                           precision    recall  f1-score   support

                    account_activate_item       0.14      0.60      0.23         5
       account_activate_item_not_possible       0.00      0.00      0.00         5
               account_ask_item_available       1.00      1.00      1.00         5
                      account_delete_item       1.00      0.80      0.89         5
                         account_get_item       0.50      1.00      0.67         5
                    account_login_problem       1.00      0.20      0.33         5
                            account_merge       1.00      0.60      0.75         5
account_setup_configure_item_not_possible       0.16      0.80      0.27         5
                        account_terminate       1.00      0.60      0.75         5
                           account_verify       1.00      0.40      0.57         5
                    account_verify_failed       1.00      0.60   

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


few_shot_simple_data
                                           precision    recall  f1-score   support

                    account_activate_item       0.17      0.60      0.26         5
       account_activate_item_not_possible       0.67      0.80      0.73         5
               account_ask_item_available       0.83      1.00      0.91         5
                      account_delete_item       1.00      1.00      1.00         5
                         account_get_item       0.38      1.00      0.56         5
                    account_login_problem       1.00      0.20      0.33         5
                            account_merge       0.00      0.00      0.00         5
account_setup_configure_item_not_possible       0.30      0.60      0.40         5
                        account_terminate       0.75      0.60      0.67         5
                           account_verify       0.50      0.40      0.44         5
                    account_verify_failed       0.60      0.60   

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m